### 재무제표 주석에서 특정 Section 찾기

* 문서 URL 목록 가져오기

In [1]:
import pandas as pd

# 사업보고서 제무제표 주석 문서 URL 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes_urls/fs_notes_url_all.csv"
fs_notes_all = pd.read_csv(file_path, dtype=object)
# 회사코드 정렬
fs_notes_all = fs_notes_all.sort_values(by=['corp_code'], ascending=True)
# 보고서 목록
list_all_codes = fs_notes_all[fs_notes_all.rpt_num.str.startswith('2022')].rpt_num.unique()
len(list_all_codes)

372

In [20]:
fs_notes_all[fs_notes_all.corp_code == '00102618']

,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,rpt_url,doc_title,doc_url,doc_result
58,00102618,012200,Y,계양전기,20220420000292,[기재정정]사업보고서 (2019.12),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,5. 재무제표 주석,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,OK
57,00102618,012200,Y,계양전기,20220420000281,[기재정정]사업보고서 (2017.12),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,5. 재무제표 주석,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,OK
56,00102618,012200,Y,계양전기,20220420000289,[기재정정]사업보고서 (2018.12),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,5. 재무제표 주석,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,OK
59,00102618,012200,Y,계양전기,20230323000476,[기재정정]사업보고서 (2022.12),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,5. 재무제표 주석,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,OK
54,00102618,012200,Y,계양전기,20220407002814,사업보고서 (2021.12),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,5. 재무제표 주석,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,OK
53,00102618,012200,Y,계양전기,20220323001202,사업보고서제출기한연장신고서 (2021.12),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,NaN,NaN,No Financial Statement Notes
52,00102618,012200,Y,계양전기,20170331000782,사업보고서 (2016.12),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,5. 재무제표 주석,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,OK
55,00102618,012200,Y,계양전기,20220420000327,[기재정정]사업보고서 (2020.12),http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,5. 재무제표 주석,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,OK


In [83]:
fs_notes_all[fs_notes_all.rpt_num == '20220408002890']['doc_url'].isna()

196    True
Name: doc_url, dtype: bool

* 리스트 분할

In [2]:
list_target_codes = list_all_codes
list_cnt = len(list_target_codes)
n = 100
list_of_lists = [list_target_codes[i * n:(i + 1) * n] for i in range((list_cnt + n - 1) // n )] 
len(list_of_lists)

4

* 재무제표 주석 문서에서 '퇴직연금' 관련 Section 영역 찾기

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# 검색 우선순위를 고려한 순서로 리스트 구성
TARGET_KEYWORDS = [
    '순확정급여부채',
    '순확정급여채무',    
    '퇴직급여부채',
    '퇴직급여제도',
    '퇴직급여채무', 
    '퇴직급여충당부채',
    '퇴직급여',
    '확정급여부채',
    '확정급여제도',
    '종업원급여'
]
        
def cleanPage(page):
    # span tag 제거
    page = page.replace("<span style=\'font-weight:bold;\'>", "")
    page = page.replace("<SPAN style=\'font-weight:bold;\'>", "")
    page = page.replace("<span style=\'color:#000000;\'>", "")
    page = page.replace("<SPAN style=\'color:#000000;\'>", "")
    page = page.replace("<span style=\'color:#000000;font-weight:bold;\'>", "")
    page = page.replace("<SPAN style=\'color:#000000;font-weight:bold;\'>", "")
    page = page.replace("<span style=\'font-size:14pt;line-height:1.6em;font-weight:normal;\'>", "")
    page = page.replace("<SPAN style=\'font-size:14pt;line-height:1.6em;font-weight:normal;\'>", "")
    page = page.replace("<span style=\'font-size:14pt;line-height:1.6em;\'>", "")
    page = page.replace("<SPAN style=\'font-size:14pt;line-height:1.6em;\'>", "")
    page = page.replace("<span style=\'font-family:DartNBSP, &quot;바탕&quot;, Batang;font-size:12pt;line-height:1.6em;\'>", "")
    page = page.replace("<SPAN style=\'font-family:DartNBSP, &quot;바탕&quot;, Batang;font-size:12pt;line-height:1.6em;\'>", "")
    page = page.replace("<span>", "")
    page = page.replace("</span>", "")
    page = page.replace("<SPAN>", "")
    page = page.replace("</SPAN>", "")
    """
    줄바꿈(Line Breaks) 제거
    \r\n - on a windows computer
    \r - on an Apple computer
    \n - on Linux
    """
    # page = page.replace("\r\n", "")
    # page = page.replace("\r", "")
    # page = page.replace("\n", "")
    # strip out all tailing whitespace
    # for line in page:
    #     line = line.rstrip()    
    # page = re.sub('\n\n', '', page)
    # page = ''.join(page.splitlines())
        
    # 줄바꿈없는 공백(Non-breaking Space) 표식 제거
    page = page.replace("&nbsp;", " ")
    # page = page.replace(u'\xa0', '')
    # page = page.replace('\xa0', ' ')
    # page = page.replace("\xc2\xa0", " ")    
    

    
    return page

def get_document(doc_url):
    # 재무제표 주석 url의 문서 가져오기
    response = requests.get(doc_url)
    if response.status_code != 200:     # URL GET '200 정상'
        return None
        
    page = cleanPage(response.text)
    soup = BeautifulSoup(page, features="lxml")
    return soup
    
def get_subtitles(soup):        
    # tags = soup.find_all('p')
    tags = soup.find_all(string = True)
    subtitle_list = []
    prev_number = 0
    for t in tags:
        # subjects = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열      
        subjects1 = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열     
        subjects2 = re.findall('\d+[.]+[^0-9]+', t.text)  # 숫자+'.'+숫자가아닌모든문자열
        subjects = subjects1 + subjects2        
        if len(subjects) > 0:
            for s in subjects:
                if not s.__contains__('주석'):
                    this_number = int(s.split(".")[0])
                    if this_number == prev_number + 1:
                        subtitle_list.append(s)
                        prev_number = this_number
    
    return subtitle_list

def find_section_title(subtitle_list):
    if len(subtitle_list) == 0:
        return None
    
    section_title = None
    for keyword in TARGET_KEYWORDS:
        for subtitle in subtitle_list:
            if subtitle.__contains__(keyword):
                section_title = subtitle
                break    

    if section_title is None:
        return None
    
    return section_title

def get_section_html(soup, subtitle_list, section_title):
    idx = 0
    idx = subtitle_list.index(section_title)
    next_title = subtitle_list[idx + 1]
    # 문서에서 해당 Section 시작위치와 종료위치 찾기
    doc_html = soup.prettify()
    p0 = doc_html.find(section_title)
    p1 = doc_html.find(next_title)
    section_html = doc_html[p0:p1]     # 문서 Slicing
    return section_html

In [5]:
import traceback
from time import sleep
from tqdm import tqdm

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/section/"

for reports in list_of_lists:
    data_list = []
    for rpt_num in tqdm(reports):
        sleep(1.0)
        df_doc = fs_notes_all.query("rpt_num == @rpt_num")
        for i, doc in df_doc.iterrows():
            doc_url = doc['doc_url']
            if isinstance(doc_url, str) and doc_url != None:
                try:
                    soup = get_document(doc_url)
                    subtitle_list = get_subtitles(soup)
                    section_title = find_section_title(subtitle_list)
                    if section_title is None:      
                        data_list.append({'corp_code' : doc['corp_code'], 'corp_name' : doc['corp_name'], 
                                        'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 'doc_title' : doc['doc_title'], 'doc_result' : doc['doc_result'], 
                                        'section_title' : None, 'doc_url' : doc_url})
                    else:
                        data_list.append({'corp_code' : doc['corp_code'], 'corp_name' : doc['corp_name'], 
                                        'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 'doc_title' : doc['doc_title'], 'doc_result' : doc['doc_result'], 
                                        'section_title' : section_title, 'doc_url' : doc_url})
                except ValueError as err:
                    print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc_url)
                    print(err)
                    break
                except Exception as e:
                    print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc_url)
                    err_msg = traceback.format_exc()
                    print(err_msg)
                    break
    
    # 결과를 파일로 저장
    if len(data_list) > 0:
        df_sections = pd.DataFrame(data_list)
        file_name = 'section_' + myutil.now_dt_str() + '.csv'
        df_sections.to_csv(path_dir + file_name, index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 72/72 [01:58<00:00,  1.65s/it]


In [6]:
import os
import pandas as pd

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/section/"
file_list = os.listdir(path_dir)

df_all = pd.DataFrame()    
for file_name in file_list:
    if file_name != 'sections_all.csv' and not file_name.startswith('__'):
        df = pd.read_csv(path_dir + file_name, dtype=object)
        df_all = pd.concat([df_all, df])
    
df_all.drop_duplicates(subset=['corp_code','rpt_num'], keep='first', inplace=True, ignore_index=True)            
df_all.to_csv(path_dir + 'sections_all.csv', index=False)

In [10]:
# pd.options.display.max_colwidth = 50       # default
pd.options.display.max_colwidth = 300

In [9]:
# Section Title을 찾지 못한 건들
df_err = df_all[df_all['section_title'].isna()]
df_err.shape[0]

40

In [11]:
df_err

,corp_code,corp_name,rpt_num,rpt_name,doc_title,doc_result,section_title,doc_url
5,00102113,경인양행,20220330000263,[기재정정]사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220330000263&dcmNo=8515531&eleId=24&offset=893391&length=363881&dtd=dart3.xsd
16,00104388,국도화학,20220310001195,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220310001195&dcmNo=8451016&eleId=22&offset=794726&length=373990&dtd=dart3.xsd
44,00111722,미래에셋증권,20220316001023,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220316001023&dcmNo=8466433&eleId=20&offset=2183904&length=1055216&dtd=dart3.xsd
47,00112059,상상인증권,20220321001270,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220321001270&dcmNo=8485923&eleId=20&offset=752092&length=420967&dtd=dart3.xsd
52,00113207,대한전선,20220322001052,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220322001052&dcmNo=8493194&eleId=22&offset=916205&length=486097&dtd=dart3.xsd
77,00121570,문배철강,20220310001184,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220310001184&dcmNo=8450948&eleId=22&offset=331367&length=448096&dtd=dart3.xsd
78,00122056,미창석유공업,20220324000331,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220324000331&dcmNo=8501290&eleId=22&offset=668365&length=244912&dtd=dart3.xsd
80,00122481,태경비케이,20220315000630,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220315000630&dcmNo=8459908&eleId=22&offset=943395&length=427086&dtd=dart3.xsd
86,00123541,보해양조,20220315001204,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220315001204&dcmNo=8462286&eleId=22&offset=855427&length=386612&dtd=dart3.xsd
93,00126089,대유플러스,20220322000990,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20220322000990&dcmNo=8492872&eleId=22&offset=1453749&length=734008&dtd=dart3.xsd


In [7]:
doc_url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=20220331003424&dcmNo=8531609&eleId=24&offset=779614&length=319129&dtd=dart3.xsd'
response = requests.get(doc_url)
page = cleanPage(response.text)
soup = BeautifulSoup(page, features="lxml")
  
tags = soup.find_all(string = True)
subtitle_list = []
prev_number = 0
for t in tags:
    print(t.text)
    subjects1 = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열     
    subjects2 = re.findall('\d+[.]+[^0-9]+', t.text)  # 숫자+'.'+숫자가아닌모든문자열
    subjects = subjects1 + subjects2
    if len(subjects) > 0:
        for s in subjects:
            if not s.__contains__('주석'):
                this_number = int(s.split(".")[0])
                if this_number == prev_number + 1:
                    subtitle_list.append(s)
                    prev_number = this_number






















5. 재무제표 주석




주석














2021년 12월 31일 현재






2020년 12월 31일 현재






삼양통상주식회사










1. 회사의 개요
삼양통상주식회사(이하 "회사", 대표이사 허남각)는 1957년 10월 12일에 피혁원단의제조, 운동용 신발의 제조 및 무역업을 영위하기 위하여 설립되었으며, 1989년 11월 10일에 한국거래소가 개설하는 유가증권시장(구:한국증권거래소)에 주식을 상장하였습니다. 회사의 본사는 서울 강남구 테헤란로에 소재하고 있으며, 당기말 현재 주요주주는 허남각 대표이사 외 (지분율 55.32%)입니다.  한편, 회사는 2005년 4월 공정거래위원회로부터 '독점규제 및 공정거래에 관한 법률'에 따라 '지에스' 기업집단에 속하는 회사로 지정되었습니다.
2. 중요한 회계정책




2.1 재무제표 작성 기준


 


회사의 재무제표는 한국채택국제회계기준(이하 기업회계기준)에 따라 작성됐습니다.한국채택국제회계기준은 국제회계기준위원회("IASB")가 발표한 기준서와 해석서 중대한민국이 채택한 내용을 의미합니다.


재무제표는 다음을 제외하고는 역사적 원가에 기초하여 작성하였습니다.
















- 


특정 금융자산과 금융부채(파생상품 포함), 공정가치로 측정하는 특정 유형자산과 투자부동산 유형






- 


순공정가치로 측정하는 매각예정자산






- 


확정급여제도와 공정가치로 측정하는 사외적립자산








한국채택국제회계기준은 재무제표 작성 시 중요한 회계추정의 사용을 허용하고 있으며, 회계정책을 적용함에 있어 경영진의 판단을 요구하고 있습니다. 보다 복잡하고 높은 수준의 판단이 필요한 부분이나 중요한 가정 및 추정이 필요한 부분은 주석2.4에서 설명하고 있습니다.




2.2 회계정책과 공시의 변경


2.2.1 회사가 채택한 제ㆍ개정 기준서 및 해석서


 


회사는 2021년 1월 1일로 개시하는 회계기간부터 

In [ ]:
<span style="font-family:DartNBSP, &quot;바탕&quot;, Batang;font-size:12pt;line-height:1.6em;">1</span>
<span style="font-family:DartNBSP, &quot;바탕&quot;, Batang;font-size:12pt;line-height:1.6em;">1. 재고자산<br></span>

In [8]:
subtitle_list

['1. 회사의 개요',
 '2. 중요한 회계정책',
 '3. 금융상품의 범주',
 '4. 금융상품의 공정가치',
 '5. 현금및현금성자산',
 '6. 장ㆍ단기금융상품',
 '7. 장ㆍ단기투자금융자산',
 '8. 매출채권 및 기타유동채권',
 '9. 기타유동자산',
 '10. 재고자산',
 '11. 종속기업 및 관계기업투자주식',
 '12. 유형자산',
 '13. 투자부동산',
 '14. 무형자산',
 '15. 기타비유동자산',
 '16. 리스',
 '17. 매입채무 및 기타채무',
 '18. 단기차입금',
 '19. 순확정급여채무(자산) ',
 '20. 보험가입의 내용',
 '21. 제공받은 보증의 내용',
 '22. 담보제공자산의 내용',
 '23. 약정사항의 내용',
 '24. 자본금',
 '25. 기타자본항목',
 '26. 이익잉여금',
 '27.  배당금 내용',
 '28. 특수관계자간 주요거래',
 '29. 판매관리비',
 '30. 기타수익',
 '31. 기타비용',
 '32. 금융수익 및 금융비용',
 '33. 외화자산ㆍ부채의 환산내용',
 '34. 비용의 성격별 분류',
 '35. 법인세회계',
 '36. 주당이익',
 '37. 현금흐름표에 관한 사항',
 '38. 환경정책 등',
 '39. 매출액 및 매출원가',
 '40. 사업부문별 공시정보',
 '41. 위험관리 목적 및 정책']

* 정규표현식으로 string에서 특정 문자열 삭제 (TEST)

In [5]:
html = "<span style=\"font-weight:bold;\"><span style=\"font-weight:bold;\">14<span style=\"font-weight:bold;\">. 무형자산<span style=\"font-weight:bold;\">"
html

'<span style="font-weight:bold;"><span style="font-weight:bold;">14<span style="font-weight:bold;">. 무형자산<span style="font-weight:bold;">'

In [6]:
# html2 = html.replace("<(span|SPAN)[^>]*>","")
html2 = html.replace("<span style=\"font-weight:bold;\">","")
html2

'14. 무형자산'

In [7]:
html3 = cleanPage(html)
html3

'14. 무형자산'

In [18]:
subtitle_list

['1. 회사의 개요',
 '2. 중요한 회계정책',
 '3. 중요한 회계추정 및 가정',
 '4. 영업부문별 정보',
 '5. 범주별 금융상품',
 '6. 금융상품 공정가치',
 '7. 현금및현금성자산',
 '8. 금융상품 및 투자자산',
 '9. 사용제한 금융상품',
 '10. 매출채권 및 기타수취채권',
 '11. 재고자산',
 '12. 기타유동자산',
 '13. 유형자산 (사용권자산 포함)',
 '14. 무형자산',
 '15. 매입채무와 기타채무',
 '16. 차입금 및 전환사채',
 '17. 충당부채',
 '18. 기타유동부채',
 '19. 퇴직급여제도',
 '20. 자본',
 '21. 수익 및 계약자산과 계약부채',
 '22. 판매비와관리비',
 '23. 비용의 성격별 분류',
 '24. 금융손익 및 기타영업외손익',
 '25. 법인세비용',
 '26. 주당손익',
 '27. 특수관계자 거래',
 '28. 현금흐름표에 관한 정보',
 '29. 우발부채 및 약정사항 등',
 '30. 위험관리',
 '31. 보고기간 후 사건']

In [4]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html style="border:0">
<head>
<title></title>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-2"><a name="toc1">5. 재무제표 주석</a></p>
<p><span style="font-weight:bold;"><br/>
</span></p>
<p><span style="font-weight:bold;">1. 회사의 개요<br/><br/>주식회사 대양금속(이하 "당사")은 1973년 8월 22일에 설립된 스테인레스 냉연강판 제조업체로서, 충청남도 예산군 신암면 추사로 146-8에 소재하고 있습니다. 당사는 1994년 4월에 주식을 한국거래소에 상장(종목코드: 009190)하였으며, 이후 수차례 증자 및 감자 등을 거쳐  당기말 현재 납입자본금은 22,459백만원(누적 참가적 우선주 자본금 6,375백만원 포함)입니다.<br/><br/>
</span></p>
<p>한편, 당기말 현재 당사의 주요 주주현황은 다음과 같습니다.</p>
<table border="1" width="630">
<colgroup>
<col width="219"/>
<col width="137"/>
<col width="137"/>
<col width="1